# In-Class Activity: Evaluating the PAVA Program

This notebook provides starter code for analyzing narratives submitted by states and territories as part of the [Protection and Advocacy for Voting Access (PAVA)](https://www.acl.gov/programs/aging-and-disability-networks/state-protection-advocacy-systems) program. These narrative reports were obtained by the Bipartisan Policy Center via a Freedom of Information Act (FOIA) request.

### Background
The PAVA program was established by the [Help America Vote Act (HAVA)](https://www.congress.gov/bill/107th-congress/house-bill/3295) of 2002, which authorized a new grant program to improve the accessibility of election systems for individuals with disabilities. **Section 291** supports nonprofit protection and advocacy systems that train, assist, and advocate for voters with disabilities, and evaluate the accessibility of voting systems and technologies.

Although recipients of these grants are required to submit annual reports on their activities, no comprehensive analysis has been done to evaluate their impact. 

Your goal is to evaluate the real-world impact of PAVA using state-reported activities from fiscal years 2020, 2021, and 2023.

# Starter code (do not edit)
- In response to our FOIA request, the Administration for Community Living (ACL) provided us with three word documents with program narratives from states for 2020, 2021, and 2023.
- The starter code provided below parses the word docs and loads the narratives into a pandas dataframe.
- Each row includes:
  - `state`: the reporting state or territory
  - `year`: the reporting year
  - `text`: the narrative description of activities

In [31]:
# import libraries
from docx import Document
import pandas as pd
import re

from openai import OpenAI
import json

The below code block loads data from the word docs into a pandas dataframe.

In [ ]:
# List of documents to import
docs = [
    "PAVA - FY 2020 Narratives.docx", 
    "PAVA - FY 2021 Narratives.docx", 
    "PAVA - FY 2023 Narratives.docx"
]

all_data = []

for doc_path in docs:
    # Extract year from filename using regex
    year_match = re.search(r'FY (\d{4})', doc_path)
    year = int(year_match.group(1)) if year_match else None

    # Load the Word document
    doc = Document(doc_path)

    # Loop through all tables in the document
    for table in doc.tables:
        for row in table.rows:
            cells = row.cells
            if len(cells) >= 2:
                state = cells[0].text.strip()
                text = cells[1].text.strip()
                if state and text:
                    all_data.append({
                        'state': state,
                        'text': text,
                        'year': year
                    })

# Combine all data into a single DataFrame
df = pd.DataFrame(all_data)

# Drop rows where state == 'P&A'
df = df[df['state'] != 'P&A']

# Group by state and year, and concatenate text
df = (
    df.groupby(['state', 'year'], as_index=False)
      .agg({'text': ' '.join})
)

# Getting started: Exploring the dataset
Begin by examining what’s in the data:
- Use `.head()` or `.sample()` to skim through different state entries.
- Plot or print out the longest and shortest narratives by word count.
- Find the most common words across all narratives or by year using `collections.Counter`.


In [ ]:
# show df
df

,state,year,text
0,AR,2021,An individual with a physical disability who u...
1,AR,2023,DRA did not work any cases funded with PAVA in...
2,AZ,2020,The Arizona Center for Disability Law’s (ACDL)...
3,AZ,2021,The Arizona Center for Disability Law’s (ACDL)...
4,AZ,2023,The Arizona Center for Disability Law’s (ACDL)...
...,...,...,...
67,WV,2020,"DRWV assisted 10 individuals, through service ..."
68,WV,2023,Nineteen PAVA funded cases were opened and sev...
69,WY,2020,PAVA staff worked with a County Clerk in testi...
70,WY,2021,PAVA collaborated with election officials rega...


# Practice integrating ChatGPT's API
Use the ChatGPT API to classify each narrative into one or more of the following impact areas:

```python
impact_areas = [
    "Voter Education & Outreach",
    "Voter Registration & Participation Initiatives",
    "Monitoring, Legal Advocacy, & Complaint Resolution",
    "Collaboration with Election Officials & Policy Influence",
    "Infrastructure & Technology Improvements"
]
```
To ensure clean, reliable output, use structured output parsing, a feature of the ChatGPT API that lets you define a JSON schema for the model to follow.

Instead of relying on freeform text, you send the model a schema describing what fields you want (e.g., 5 Booleans for the impact areas), and the model returns a response that conforms to that structure. This avoids the need to post-process or “extract” values from raw text, making your pipeline much more robust.

This is the method Professor Adler demonstrated in class last week. You might consult the following resources to help you:
* [OpenAI Structured Outputs Documentation](https://platform.openai.com/docs/guides/structured-outputs/introduction?context=ex2&api-mode=responses)
* [Professor Adler's Example](https://github.com/Computer-Programming-for-Lawyers/Spring-2025/blob/main/lecture/week-8/week8_part2.ipynb)

**TLDR: Classify each activity based on whether it performed the following categories of activities. The result should be the same pandas dataframe with a column for each impact area. The values of the colum should be boolean (True/False) indicating whether the narrative performs a task within that impact area.**

In [ ]:
impact_areas = [
    "Voter Education & Outreach",
    "Voter Registration & Participation Initiatives",
    "Monitoring, Legal Advocacy, & Complaint Resolution",
    "Collaboration with Election Officials & Policy Influence",
    "Infrastructure & Technology Improvements"
]

In [ ]:
# Prof. Orey API key for today's activity (subject to limits)
API_KEY = "hidden"

# Post-classification analysis

Once each narrative has been categorized:
* Count how many states reported work in each impact area.
* Compare how activity types varied by year.
* Identify states that reported across all five categories.
* Visualize trends over time (e.g., line charts of impact area frequency by year).
* Explore regional differences by mapping activity types across the U.S.
